In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import matplotlib as mpl
import matplotlib.font_manager as fm

# 폰트 파일 경로 지정
font_path = '/content/drive/MyDrive/font/NanumGothic.ttf'

# 폰트 등록 및 matplotlib 설정
font_name = fm.FontProperties(fname=font_path).get_name()
fm.fontManager.addfont(font_path)
mpl.rc('font', family=font_name)

In [ ]:
import pandas as pd
from google.colab import drive, files

# 1. Google Drive 마운트
drive.mount('/content/drive')

# 2. 파일 경로 설정 (사용자 지정 경로)
data_root = '/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling'
file_path = f'{data_root}/train_12.parquet'

# # 3. Parquet 파일 불러오기
# df = pd.read_parquet(file_path)

# # 4. CSV로 저장 (엑셀용 UTF-8-sig 인코딩)
# csv_filename = 'train_12.csv'
# df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

# # 5. 로컬로 다운로드
# files.download(csv_filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 파일 경로 설정
data_root = '/content/drive/MyDrive/Colab Notebooks/final_project/final_sampling'

X_path = f'{data_root}/train_12.parquet'
y_path = f'{data_root}/segment_12.parquet'

# 데이터 불러오기
X_df = pd.read_parquet(X_path)
y_df = pd.read_parquet(y_path)

# ID 기준으로 병합
df = pd.merge(X_df, y_df, on='ID', how='inner')

# 확인
print(df.shape)
df.head()

(400000, 313)


,기준년월,ID,이용금액_R3M_신용체크,입회경과개월수_신용,_1순위카드이용금액,회원여부_이용가능_카드론,이용거절여부_카드론,최종카드발급경과월,이용금액_R3M_신용,_1순위카드이용건수,...,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M,잔액_한도소진율,증감율_카드론_분기,증감율_일시불_분기,증감율_체크_분기,증감율_카드론_전월,변동률_RV평잔,Segment
0,201812,TRAIN_000000,-454,71,3027,0,0,27,-454,25,...,0.000000,0.878859,1.398627,3.407027,0.143423,-0.609648,0.086480,0.048726,0.321733,D
1,201812,TRAIN_000001,7089,16,7259,1,0,23,7089,31,...,0.000000,0.000000,0.000000,5.196375,0.143423,-0.597498,0.086480,0.048726,0.214949,E
2,201812,TRAIN_000002,27336,128,26968,0,0,25,27336,52,...,-0.115879,0.187467,-1.198788,4.332080,0.143423,-0.154887,0.086480,0.048726,-2.839702,C
3,201812,TRAIN_000003,4270,31,4807,0,0,22,4270,27,...,0.000000,0.781401,1.282494,6.248569,0.143423,0.593422,0.086480,0.048726,0.321733,D
4,201812,TRAIN_000004,9385,6,3989,1,0,20,7387,-2,...,0.000000,0.762016,0.986860,-2.056257,0.143423,4.964378,-0.654283,0.048726,0.321733,E


In [ ]:
# 타겟 변수와 피처 나누기
df = df.drop(columns=['ID', '기준년월'])
X = df.drop(columns=['Segment'])
y = df['Segment']

In [ ]:
# Label Encoder
from sklearn.preprocessing import LabelEncoder

# 타겟 값 y 인코딩
le_y = LabelEncoder()
y = le_y.fit_transform(y)

In [ ]:
import joblib

# LabelEncoder 객체를 .dat 이름으로 저장 (실제로는 joblib 포맷)
joblib.dump(le_y, "label_encoder_segment.dat")


# from google.colab import files
# files.download("label_encoder_segment.dat")

# # 불러올 때
# le_loaded = joblib.load("label_encoder_segment.dat")


['label_encoder_segment.dat']

##변수명 test -> val !!!!!!!!!!! 추후 test 데이터 불러올 때 헷갈리니까

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
# import joblib
# # prompt: scaler 를 따로 저장해서 내 컴퓨터에 다운로드

# # Scaler 저장
# joblib.dump(scaler, "standard_scaler.pkl")

# # Scaler 다운로드
# files.download("standard_scaler.pkl")

전처리 완료 변수들
- X, y: 병합 및 인코딩된 전체 데이터셋
- le_y: y(LabelEncoder 객체)
- X_train_scaled, X_val_scaled, y_train, y_val: 스케일링된 train/val 데이터셋

# Segment 'E' vs 나머지 (XGBoost) - 제갈서현

E라고 예측한 게 너무 많아서.. 1단계에서 정확도를 좀 낮추자. E가 아닌 걸 더 잘 맞춰야 함.
accuracy는 손해봐도 된다. 근데 올라가야 한다. 모델 다시 학습시켜야 한다. weighted

In [ ]:
# Segment 'E' 분류용 이진 타겟 생성
target_E = le_y.transform(['E'])[0]
y_train_step1 = (y_train == target_E).astype(int)
y_val_step1 = (y_val == target_E).astype(int)

In [ ]:
# from xgboost import XGBClassifier
# import numpy as np

# # Step 1: E vs Others 분류용 라벨 생성
# y_train_step1 = (y_train == le_y.transform(['E'])[0]).astype(int)
# y_val_step1 = (y_val == le_y.transform(['E'])[0]).astype(int)

# model_step1 = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
# model_step1.fit(X_train_scaled, y_train_step1)

# # 예측: 전체 test set에 대해 E 확률
# prob_E = model_step1.predict_proba(X_val_scaled)[:, 1]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:38:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


1. XGBoost, LGBM, CatBoost 각 돌려

In [ ]:
!pip install lightgbm

In [ ]:
!pip install catboost

In [ ]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier

import joblib  # 혹시 다른 모델도 저장하고 싶을 경우 대비

# 모델 정의
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(random_state=42)
cat_model = CatBoostClassifier(verbose=0, random_state=42)  # 학습용 CatBoost 객체

models = {'XGBoost': xgb, 'LightGBM': lgbm, 'CatBoost': cat_model}

tuned_cat = CatBoostClassifier(
    depth=6, learning_rate=5e-3,
    early_stopping_rounds=20,
    class_weights=[10, 1],
    iterations=100,
    random_state=42,
    verbose=0
)
tuned_cat.fit(X_train_scaled, y_train_step1)



In [ ]:
# 평가
y_pred = tuned_cat.predict(X_val_scaled)
print("Tuned CatBoost - F1:", f1_score(y_val_step1, y_pred))

# 저장
tuned_cat.save_model("model_step1_catboost_tuned.dat")

Tuned CatBoost - F1: 0.8454088317574985


In [ ]:
from google.colab import files

files.download("model_step1_catboost_tuned.dat")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

| 모델           | Accuracy   | F1 Score             |
| ------------ | ---------- | -------------------- |
| XGBoost      | 0.9370     | 0.9610               |
| LightGBM     | 0.9290     | 0.9561               |
| **CatBoost** | **0.9417** | **0.9639** ✅ (최고 성능) |


In [ ]:
10/0

2. Soft, Hard Voting 해보기(XGBoost, LGBM, CatBoost)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, f1_score

# 모델 재정의 (훈련 이전 상태로 초기화)
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
lgbm = LGBMClassifier(random_state=42)
cat = CatBoostClassifier(verbose=0, random_state=42, allow_writing_files=False)

# Hard Voting
hard_voting = VotingClassifier(
    estimators=[('xgb', xgb), ('lgbm', lgbm), ('cat', cat)],
    voting='hard'
)
hard_voting.fit(X_train_scaled, y_train_step1)
y_pred_hard = hard_voting.predict(X_val_scaled)

# Soft Voting
soft_voting = VotingClassifier(
    estimators=[('xgb', xgb), ('lgbm', lgbm), ('cat', cat)],
    voting='soft'
)
soft_voting.fit(X_train_scaled, y_train_step1)
y_pred_soft = soft_voting.predict(X_val_scaled)

# 성능 비교
print("===== Hard Voting =====")
print(f"Accuracy: {accuracy_score(y_val_step1, y_pred_hard):.4f}")
print(f"F1 Score:  {f1_score(y_val_step1, y_pred_hard):.4f}")

print("\n===== Soft Voting =====")
print(f"Accuracy: {accuracy_score(y_val_step1, y_pred_soft):.4f}")
print(f"F1 Score:  {f1_score(y_val_step1, y_pred_soft):.4f}")

| 모델          | Accuracy   | F1 Score     |
| ----------- | ---------- | ------------ |
| XGBoost     | 0.9370     | 0.9610       |
| LightGBM    | 0.9290     | 0.9561       |
| **CatBoost**    | 0.9417     | **0.9639** ✅ |
| Hard Voting | 0.9385     | 0.9620       |
| Soft Voting | **0.9386** | 0.9620       |

In [ ]:
from catboost import CatBoostClassifier

# 저장된 .dat 모델 로드
cat_loaded = CatBoostClassifier()
cat_loaded.load_model("model_step1_catboost.dat")

| 메서드               | 역할                   | 결과                         | 언제 쓰는지                   |
| ----------------- | -------------------- | -------------------------- | ------------------------ |
| `predict()`       | **최종 예측 클래스**를 반환    | `0`, `1` 또는 `A`, `B` 같은 라벨 | 분류 결과가 뭔지 알고 싶을 때        |
| `predict_proba()` | **각 클래스에 속할 확률**을 반환 | `[0.2, 0.8]`처럼 확률 값        | 확률 기반 판단, threshold 조정 시 |


3. CatBoost + Optuna (F1 최적화) - 쓰지마(우리 방법부터)

In [ ]:
!pip install optuna

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Optuna 목적 함수 정의
def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'iterations': trial.suggest_int('iterations', 100),
        'random_seed': 42,
        'verbose': 0,
        'task_type': 'CPU'
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train_scaled, y_train_step1)
    y_pred = model.predict(X_val_scaled)
    return f1_score(y_val_step1, y_pred)

# 스터디 생성 및 튜닝 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # n_trials 수 늘릴수록 더 정밀해짐

# 최적 파라미터 확인
print("Best trial:")
print(study.best_trial)

4. 최적 파라미터로 CatBoost 학습 & 저장

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score

# 최적 파라미터로 모델 학습
best_params = {
    'depth': 7,
    'learning_rate': 0.25350491503956507,
    'l2_leaf_reg': 7.447977718157723,
    'iterations': 409,
    'random_seed': 42,
    'verbose': 0
}

# 모델 학습
cat_best = CatBoostClassifier(**best_params)
cat_best.fit(X_train_scaled, y_train_step1)

# 예측 및 성능 확인
y_pred = cat_best.predict(X_val_scaled)
print("최적 CatBoost 성능")
print("Accuracy:", accuracy_score(y_val_step1, y_pred))
print("F1 Score:", f1_score(y_val_step1, y_pred))

# 모델 저장 (.dat 형식)
cat_best.save_model("model_step1_catboost_tuned.dat")
print("저장 완료: model_step1_catboost_tuned.dat")

confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 예측값 얻기
y_pred = cat_best.predict(X_val_scaled)

# Confusion Matrix 계산
cm = confusion_matrix(y_val_step1, y_pred)

# 시각화
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not E", "E"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Segment E vs Others")
plt.show()

  |               | 예측: Not E   | 예측: E       |
| ------------- | ----------- | ----------- |
| **실제: Not E** | 13,057 (TN) | 2,875 (FP)  |
| **실제: E**     | 1,850 (FN)  | 62,218 (TP) |
- E 고객을 굉장히 잘 맞추고 있음 (재현율 97.1%)

- E가 아닌 고객을 E라고 착각한 경우도 조금 있음 (2,875명 → Precision 95.6%)